In [1]:
import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document 
from sentence_transformers import SentenceTransformer
import chromadb
from typing import List

## Vector Database

### Creating the Vector DB `RUN ONCE`

`Creating sentence out of the CSV`

In [ ]:
csv_path = r"C:\Users\dana.alnadi\Documents\Visual Studio 2022\Code Snippets\Python\HRChatbot\data\csvs\medical_network_cleaned.csv"

df = pd.read_csv(csv_path)

documents = []

name_col = "Name"
loc_col = "location"
phone_col = "Phone"
main_cat_col = "Main Category"
subcat_col = "Sub Category"

for _, row in df.iterrows():
    name = str(row[name_col]).strip()
    loc = str(row[loc_col]).strip()
    phone = str(row[phone_col]).strip()
    main_cat = str(row[main_cat_col]).strip()
    subcat = str(row[subcat_col]).strip()

    sentence = (
        f"{name} هو جهة في فئة {main_cat} ضمن {subcat} "
        f"ويقع في منطقة {loc} ويمكن التواصل عبر الرقم {phone}."
    )
    documents.append(sentence)

print(f"✅ Created {len(documents)} documents with full Arabic details!")

✅ Created 4905 documents with full Arabic details!


`Create and save embeddings in Chroma using Sentence Transformer` 

In [ ]:
# Initialize model and Chroma client
model_name = 'Snowflake/snowflake-arctic-embed-l-v2.0'
model = SentenceTransformer(model_name)

# Generate embeddings
embeddings = model.encode(documents , normalize_embeddings=True) #the model uses Euclidean (L2) distance instead of cosine so we normalize the embeddings

# Connect to (or create) a local persistent Chroma database
client = chromadb.PersistentClient(path="./my_chroma_db")

# Create or get a collection
collection = client.get_or_create_collection(name="medical_network_collection")

# Add documents + embeddings
collection.add(
    ids=[f"doc_{i}" for i in range(len(documents))],
    embeddings=embeddings,
    documents=documents
)

print("✅ Embeddings saved successfully to local Chroma database.")


### Lodeing the VDB

In [3]:
def load_chroma_collection(db_path="./my_chroma_db", collection_name="medical_network_collection"):
    client = chromadb.PersistentClient(path=db_path)
    return client.get_collection(name=collection_name)

collection = load_chroma_collection(db_path="./my_chroma_db", collection_name="medical_network_collection")
print(f"✅ Loaded collection '{collection.name}' with {collection.count()} documents.")

✅ Loaded collection 'medical_network_collection' with 4905 documents.


In [4]:
stored = collection.get(include=["documents", "embeddings"])
document_embeddings = stored["embeddings"]
documents = stored["documents"]
#documents 

`Testing the embeddings`

In [ ]:
def embed_queries(queries ,model):
    query_embeddings = model.encode(queries).tolist()

    # Query Chroma collection directly
    results = collection.query(
        query_embeddings=query_embeddings,
        n_results = 10 # number of top matches per query
    )

    # Display the results
    for i, query in enumerate(queries):
        print(f"\n🔍 Query: {query}")
        docs = results["documents"][i]
        scores = results["distances"][i]
        for doc, score in zip(docs, scores):
            print(f"Score: {score:.4f} | {doc}")

    return results  

In [ ]:
# Encode the queries
model_name = 'Snowflake/snowflake-arctic-embed-l-v2.0'

model = SentenceTransformer(model_name)

queries = ["دكتور جلديه في الكرك", "دكتور أطفال في الزرقاء"]

embed_queries(queries ,model)


🔍 Query: دكتور جلديه في الكرك
Score: 0.9794 | مركز الكرك للعلاج الطبيعي هو جهة في فئة مراكز متخصصة ضمن مراكز العلاج الطبيعي ويقع في منطقة الكرك ويمكن التواصل عبر الرقم 2351447.
Score: 1.0669 | مختبرات الكرك للتحاليل الطبية هو جهة في فئة مختبرات ضمن المختبرات المعتمدة ويقع في منطقة الكرك ويمكن التواصل عبر الرقم 2353545.
Score: 1.1053 | مركز الغيمة الطبي هو جهة في فئة أطباء ضمن الطب العام ويقع في منطقة الكرك ويمكن التواصل عبر الرقم 0798226322.
Score: 1.1110 | خلدون علاوي هو جهة في فئة أطباء ضمن الامراض الداخلية (الباطنية) ويقع في منطقة الكرك ويمكن التواصل عبر الرقم 0795880580.
Score: 1.1117 | ليث عبيدات هو جهة في فئة أطباء ضمن الامراض الصدرية ويقع في منطقة الكرك ويمكن التواصل عبر الرقم 0798280150.
Score: 1.1210 | ملاك راتب صبحي النابلسي هو جهة في فئة أطباء ضمن الامراض الجلدية والتناسلية ويقع في منطقة الكرك ويمكن التواصل عبر الرقم 0799197232.
Score: 1.1250 | صيدلية الكرك الجديدة هو جهة في فئة صيدليات ضمن الصـيدليـات المعتمـدة ويقع في منطقة الكرك ويمكن التواصل عبر الرقم 2352225.
Score: 1.

{'ids': [['doc_4686',
   'doc_4566',
   'doc_231',
   'doc_568',
   'doc_2246',
   'doc_1681',
   'doc_4146',
   'doc_1970',
   'doc_471',
   'doc_4142'],
  ['doc_1385',
   'doc_1387',
   'doc_1389',
   'doc_1395',
   'doc_1390',
   'doc_1392',
   'doc_1393',
   'doc_1396',
   'doc_1386',
   'doc_1394']],
 'distances': [[0.9794231057167053,
   1.066909909248352,
   1.105273723602295,
   1.1109833717346191,
   1.1117273569107056,
   1.1210353374481201,
   1.1249890327453613,
   1.1353000402450562,
   1.1382639408111572,
   1.1450440883636475],
  [1.020609974861145,
   1.0285881757736206,
   1.065293550491333,
   1.0667290687561035,
   1.0719636678695679,
   1.0851032733917236,
   1.105173110961914,
   1.1399930715560913,
   1.1540724039077759,
   1.1593173742294312]],
 'metadatas': [[None, None, None, None, None, None, None, None, None, None],
  [None, None, None, None, None, None, None, None, None, None]],
 'embeddings': None,
 'documents': [['مركز الكرك للعلاج الطبيعي هو جهة في فئة مر

## RAG Chain

In [7]:
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA

In [8]:
#Test the LLM language support
llm = Ollama(model="command-r7b-arabic")   

print(llm("هل تفهم العربيه؟" ))

c:\Users\dana.alnadi\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


نعم، أنا أستطيع فهم اللغة العربية. أنا مصمم لمساعدة المستخدمين باللغات المختلفة، بما في ذلك اللغة العربية. إذا كان لديك أي أسئلة أو استفسارات باللغة العربية، يمكنني مساعدتك وتقديم الإجابات المناسبة.


In [12]:
embedding_function = HuggingFaceEmbeddings(model_name='Snowflake/snowflake-arctic-embed-l-v2.0')
vectordb = Chroma(
    collection_name="medical_network_collection",
    persist_directory="./my_chroma_db",
    embedding_function=embedding_function
)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k":10 })

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [14]:
query = "المختبرات المعتمدة"
response = qa_chain(query)
print("Answer:", response["result"])

c:\Users\dana.alnadi\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Answer: هناك العديد من المختبرات الطبية المعتمدة في مختلف المناطق في الأردن، وتضم هذه القائمة:

1. المؤكدة للمختبرات الطبية - منطقة مرج الحمام، رقم التواصل: 0770041416.
2. اسم المختبر - منطقة العنوان، رقم الهاتف غير متوفر.
3. المختبرات الطبية التخصصية - منطقة الزرقاء، رقم التواصل: 3993446.
4. المختبرات المتميزة للتحاليل الطبية - منطقة جبل الحسين، رقم التواصل: 5695190.
5. مختبرات العلمية الطبية - منطقة الكرك، رقم التواصل: 2353901.
6. مختبرات فلسطين للتحاليل الطبية - منطقة الوحدات، رقم التواصل: 4786906.
7. مختبرات العقبة الطبية - منطقة العقبة، رقم التواصل: 2016396.
8. المختبرات الذكية للتحاليل الطبية - منطقة الدوار الخامس، رقم التواصل: 5936241.
9. المختبرات الطبية الدقيقة - منطقتان (شارع وصفي التل والفحيص)، رقم التواصل: 5620606 أو 4624600.
10. مختبرات الجامعة الطبية - منطقة إربد، رقم التواصل: 7244095.
11. مختبرات مكه للتحاليل الطبية - شارع مكة، رقم التواصل: 5521712.
12. مختبرات الخبراء الطبية - منطقة الرصيفة، رقم التواصل: 3752255.
13. مختبرات الأردن الطبية المتخصصة - منطقة جبل الحسين، رق

In [16]:
query = "دكتور جلديه في الكرك ن"
response = qa_chain({"query": query})   # or qa_chain(query) depending on API
print(response["result"])               # or response["answer"]


ملاك راتب صبحي النابلسي هو جهة في فئة أطباء ضمن الامراض الجلدية والتناسلية ويقع في منطقة الكرك ويمكن التواصل عبر الرقم 0799197232.
